# Report of Project 1: Navigation

## Learning algorithm

The algorithm used in this project is a __dueling double DQN__ (DuelingDDQN) agent, which is described in the following sections.

The DuelingDDQN algorithm refers to a class of RL algorithms called temporal difference (TD) learning. It is a _model-free_ algorithm, because there is no prior planning or implementation of a (stochastic) model of the environment. It is an _off-policy_ algorithm, because it improves a policy different from the policy used to generate the data. While the definition of model-free should be pretty straight forward, the off-policy part may need some some further explanation and what is the definition of a policy after all? 

Simply put, a policy is a function (incorporating a neural network in our case) which maps states to actions. __The policy__ used in this project is $\epsilon$-greedy, which selects actions either uniformly at random or based on the learned policy (i.e. a Q-network). The choice between these two is based on $\epsilon$ which is decayed on each episode by a certain factor, i.e. favoring exploration in the beginning of the learning process and exploitation of the learned policy towards the end. The term off-policy refers to the fact that we use two disinct networks (same architecture but different weight-instances) for a) generating the next step's action (value) and b) updating the policy.

Every single step taken by the agent in the environment yields a so-called _experience_, which comprises a _state_ vector (e.g. representing a ray-based perception of the environment), the taken _action_ (based on the policy described above), a reward returned by the environment, the _next state_ of the agent and an indicator if the episode has finished. Each experience is stored in a fixed-size __experience replay buffer__ (a double-ended queue), from which the algorithm samples batches _uniformly at random_ to fit the neural network(s). This breaks the temporal correlation of experiences and thus generates i.i.d. samples from the buffer, which is expected by gradient-based optimizers such as Adam, RMSProp and others.

The model-fitting process (i.e. update of the neural network's weights) is done every n-th step of the agent, using a sampled batch of experiences. The process is defined by the name __double DQN__, which is made up by a vanilla _DQN_, i.e. two neural networks, the online network which is continuously updated, and the target network, which is used to generate the next step's action value. The extension _double_ refers to the fact that the _online network_ is used to choose the action (by the highest value) from the next state, as opposed to the vanilla DQN algorithm alone, where the target network is responsible for determining the max. actions _and_ it's value. 

The update step looks as follows: for all experiences in a given batch we compute the expected action values of the next state's from the online and target network, and estimates of the actual state's values from the online network. Since the experiences also contain the rewards, we can compute the gradient update as follows:

$$\nabla_{\theta_i}L_i(\theta_i) = \mathbb{E}_{(s,a,r,s')\sim U(D)} \Big[\big(r+\gamma Q(s',\underset{a'}{\operatorname{argmax}} Q(s',a';\theta_i);\theta^-) - Q(s,a;\theta_i))\nabla_{\theta_i}Q(s,a;\theta_i\big)\Big]$$

The estimated Q-values from the target (multiplied by a constant factor $\gamma$ and added reward $r$), and the local Q-value estimate, $Q(s,a;\theta_i)$, are passed on the loss function, where the `SmoothL1Loss` a.k.a. Huber-Loss has been chosen, which is more robust to extreme values, i.e. mitigating very large gradients which potentially lead to oszillations and thus slow convergence, especially in the beginning of the learning process, as the network policy is essentially chasing a moving target.

The final step in the model-fitting process is to "soft-update" the target network's weights. In contrast to overwriting the target-network weights with the online network weights every N time steps, we use __Polyak Averaging__, which updates the weights more often by mixing the weights with tiny bits of both networks:

$$\theta_i^- = \tau \theta_i + (1-\tau)\theta_i^-$$

The neural network architecture itelf is defined as __dueling DQN__, which main purpose is __sample efficiency__. The main difference to the vanilla DQN is that the layer before the output is split into two streams, where the first stream represents the state-value function $V(s)$ and the second stream represents the action-advantage function $A(s,a)$. 

Finally, reconstruction of the action-value function $Q(s,a)$ is done by combining the output of the two stream in the following way:

$$Q(s,a;\theta,\alpha,\beta) = V(s;\theta,\beta) + \Big(A(s,a;\theta,\alpha) - \frac{1}{|A|}\underset{a'} \sum A(a,s';\theta,\alpha) \Big)$$

In the equation above, $\theta$ represents the weights of the shared upstream hidden layers, $\beta$ the weights of the value-function stream and $\alpha$ the weights of the action-advantage-function stream. The rightmost term in that equation subtracts the mean of the aggregated action-value function, shifting the estimates of the action-advantage stream in order to _stabilize the optimization_ while keeping the relative rank of $A(s,a)$.

Completing this fairly concise explanation of the algorithm (with great support of the outstanding references listed below), the following table provides a listing of hyper-parameters used for the final results presented in the next section:

Parameter Name | Value | Description
:--- | --- | :---
hidden_layer_dimensions | (512, 256) | dimensions of neural network hidden layers.
activation_fn | ReLU | the activation function used for the neural network hidden layers.
buffer_size | 100_000 | replay buffer size.
batch_size | 64 | mini-batch size.
gamma | 0.99 | discount factor.
tau | 1e-3 | interpolation parameter for target-network weight update.
lr | 5e-4 | learning rate.
update_every | 4 | how often (time steps) to update the network.
n_episodes | 2000 | maximum number of training episodes.
max_t | 1000 |  maximum number of time steps per episode.
eps_start | 1.0 | starting value of epsilon, for epsilon-greedy action selection.
eps_end | 0.01 | minimum value of epsilon.
eps_decay | 0.995 | multiplicative factor (per episode) for decreasing epsilon.
scores_window_length | 100 | length of scores window to monitor convergence.
average_target_score | 13.0 | average target score for scores_window_length at which learning stops.

## Results

While the evironment could have been solved within ~600 episodes by using a vanilla DQN, extending the algorithm using a dueling architecture, the environment could be __solved in 477 episodes__.

![Scores](scores.png "Scores")

## Ideas for future work

From a modeling point of view, there are many ways to improve the algorithms and models even further. A (incomplete) listing of possible future improvements/extensions:

* Prioritized Experience Replay (PER)
* Distributional DQN
* Noisy DQN
* Rainbow DQN

Also, it would be interesting to see how Bayesian variants of the given neural network architectures (e.g. Variational Inference or Bayesian approximation using Dropout) could possibly improve decision making, based on posterior-predictive distributions.

Utilize hyper-parameter optimization frameworks (hyperopt or scikit-optimize) to further optimize the agent's settings.

From an architectural perspective, loose coupling between the `DuelingDDQN` agent, the `DuelingDenseQNetwork`s, `ReplayBuffer`, `Adam` optimizer, `SmoothL1Loss`, etc. should be preferred to enable composing and testing different implementations without changing the code. Moreover, the $\epsilon$-greedy policy code should be moved into it's own class.

## Additional References

* [Miguel Morales, "Grokking Deep Reinforcement Learning" Manning Publications.](https://www.manning.com/books/grokking-deep-reinforcement-learning)
* [Sutton, Barto "Reinforcement Learning, 2nd edition" MIT Press.](https://s3-us-west-1.amazonaws.com/udacity-drlnd/bookdraft2018.pdf)
* [Mnih, Volodymyr, et al. "Human-level control through deep reinforcement learning." Nature518.7540 (2015): 529.]( http://www.davidqiu.com:8888/research/nature14236.pdf)